<a href="https://colab.research.google.com/github/Rakeshatla/Folder-management-system/blob/main/folder_management_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 69.6 MB/s eta 0:00:00


In [20]:
import os
import fitz  # PyMuPDF
import IPython.display as display
import shutil

class Folder:
    def __init__(self, name):
        self.name = name
        self.subfolders = []
        self.files = {}

    def add_subfolder(self, folder):
        self.subfolders.append(folder)

    def remove_subfolder(self, folder_name):
        self.subfolders = [f for f in self.subfolders if f.name != folder_name]

    def find_subfolder(self, folder_name):
        for folder in self.subfolders:
            if folder.name == folder_name:
                return folder
        return None

    def add_file(self, file_name, file_path):
        self.files[file_name] = file_path

    def open_file(self, file_name):
        file_path = self.files.get(file_name)
        print('file path', file_path)
        if not file_path or not os.path.isfile(file_path):
            print(f"File '{file_name}' not found in '{self.name}' folder or path is invalid.")
            return

        if file_name.endswith(".txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                print(file.read())
        elif file_name.endswith(".pdf"):
            with fitz.open(file_path) as pdf:
                for page_num in range(len(pdf)):
                    page = pdf.load_page(page_num)
                    pix = page.get_pixmap()
                    img_path = f"/content/page_{page_num + 1}.png"
                    pix.save(img_path)
                    display.display(display.Image(img_path))
        else:
            print(f"Unsupported file format for '{file_name}'.")

    def display_contents(self):
        print(f"Contents of '{self.name}':")
        if self.subfolders:
            print("Subfolders:")
            for subfolder in self.subfolders:
                print(f"  - {subfolder.name}")
        else:
            print("No subfolders.")

        if self.files:
            print("Files:")
            for file_name in self.files.keys():
                print(f"  - {file_name}")
        else:
            print("No files.")


class FolderTree:
    def __init__(self, root_name="BRANCH"):
        self.root = Folder(root_name)
        self.opened_files = []

    def make_dir_on_disk(self, path):
        dir_path = os.path.join('/content', *path)
        os.makedirs(dir_path, exist_ok=True)
        return dir_path

    def add_folder(self, path, folder_name):
        parent = self._traverse_path(path)
        if parent:
            if parent.find_subfolder(folder_name):
                print(f"Folder '{folder_name}' already exists under '{parent.name}'.")
                return
            new_folder = Folder(folder_name)
            parent.add_subfolder(new_folder)
            dir_path = self.make_dir_on_disk(path + [folder_name])
            print(f"Folder '{folder_name}' added under '{parent.name}'.")
            print(f"Directory created on disk at: {dir_path}")
        else:
            print("Path not found.")

    def delete_folder(self, path, folder_name):
        parent = self._traverse_path(path)
        if parent:
            parent.remove_subfolder(folder_name)
            print(f"Folder '{folder_name}' removed from '{parent.name}'.")
        else:
            print("Path not found.")

    def search_folder(self, folder_name):
        result = self._dfs_search(self.root, folder_name)
        if result:
            print(f"Folder '{folder_name}' found.")
        else:
            print(f"Folder '{folder_name}' not found.")

    def add_file_to_folder(self, path, file_name, file_path):
        parent = self._traverse_path(path)
        if not parent:
            print("Path not found.")
            return
        if not os.path.isfile(file_path):
            try:
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write("")
                print(f"File '{file_path}' did not exist, but was created empty.")
            except Exception as e:
                print(f"Could not create file '{file_path}': {e}")
                return
        parent.add_file(file_name, file_path)
        print(f"File '{file_name}' added to '{parent.name}'.")

    def upload_and_add_file(self, path):
        from google.colab import files
        parent = self._traverse_path(path)
        if parent:
            uploaded = files.upload()
            for file_name, data in uploaded.items():
                file_path = os.path.join('/content', file_name)
                with open(file_path, 'wb') as f:
                    f.write(data)
                parent.add_file(file_name, file_path)
                print(f"File '{file_name}' uploaded and added to '{parent.name}'.")
        else:
            print("Path not found.")

    def open_file_in_folder(self, path, file_name):
        folder = self._traverse_path(path)
        if folder:
            folder.open_file(file_name)
            self.opened_files.append(file_name)
        else:
            print("Path not found.")

    def show_last_opened_file(self):
        if self.opened_files:
            print(f"Last opened file: '{self.opened_files[-1]}'")
        else:
            print("No files have been opened recently.")

    def open_folder(self, path):
        folder = self._traverse_path(path)
        if folder:
            folder.display_contents()
        else:
            print("Path not found.")

    def _traverse_path(self, path):
        current = self.root
        for folder_name in path:
            current = current.find_subfolder(folder_name)
            if not current:
                return None
        return current

    def _dfs_search(self, root, folder_name):
        if root.name == folder_name:
            return root
        for subfolder in root.subfolders:
            result = self._dfs_search(subfolder, folder_name)
            if result:
                return result
        return None

    def display_tree(self):
        def _display(folder, indent=0):
            print(" " * indent + folder.name)
            for file_name in folder.files:
                print(" " * (indent + 4) + f"[File] {file_name}")
            for subfolder in folder.subfolders:
                _display(subfolder, indent + 4)

        print("Folder Structure:")
        _display(self.root)

    def add_file_from_local_disk(self, folder_path, file_name, local_file_path):
        parent_folder = self._traverse_path(folder_path)
        if parent_folder is None:
            print("Target folder path not found in FolderTree.")
            return
        dest_dir = os.path.join('/content', *folder_path)
        os.makedirs(dest_dir, exist_ok=True)
        dest_file_path = os.path.join(dest_dir, file_name)
        try:
            shutil.copy2(local_file_path, dest_file_path)
            print(f"File copied to: {dest_file_path}")
        except Exception as e:
            print(f"Error copying file: {e}")
            return
        parent_folder.add_file(file_name, dest_file_path)
        print(f"File '{file_name}' added to folder '{parent_folder.name}'.")

    def menu(self):
        while True:
            print("\nMenu:")
            print("1: Add Folder")
            print("2: Delete Folder")
            print("3: Search Folder")
            print("4: Add File")
            print("5: Open File")
            print("6: Show Last Opened File")
            print("7: Display Folder Tree")
            print("8: Add File from Local Disk")
            print("0: Exit")
            choice = input("Enter your choice: ")

            match choice:
                case '1':
                    path_str = input("Enter path where to add folder (comma-separated folder names, e.g. rakesh,subfolder). Leave empty for root: ")
                    path = [p.strip() for p in path_str.split(',') if p.strip()] if path_str else []
                    folder_name = input("Enter new folder name: ")
                    self.add_folder(path, folder_name)

                case '2':
                    path_str = input("Enter path where to delete folder (comma-separated): ")
                    path = [p.strip() for p in path_str.split(',') if p.strip()] if path_str else []
                    folder_name = input("Enter folder name to delete: ")
                    self.delete_folder(path, folder_name)

                case '3':
                    folder_name = input("Enter folder name to search: ")
                    self.search_folder(folder_name)

                case '4':
                    path_str = input("Enter path to add file in (comma-separated), leave empty for root: ")
                    path = [p.strip() for p in path_str.split(',') if p.strip()] if path_str else []
                    file_name = input("Enter file name: ")
                    file_path = input("Enter actual file path on disk (e.g. /content/yourfile.txt): ")
                    self.add_file_to_folder(path, file_name, file_path)

                case '5':
                    path_str = input("Enter path to folder containing the file (comma-separated), leave empty for root: ")
                    path = [p.strip() for p in path_str.split(',') if p.strip()] if path_str else []
                    file_name = input("Enter file name to open: ")
                    self.open_file_in_folder(path, file_name)

                case '6':
                    self.show_last_opened_file()

                case '7':
                    self.display_tree()

                case '8':
                    path_str = input("Enter folder path to add file in (comma-separated, leave empty for root): ")
                    folder_path = [p.strip() for p in path_str.split(',') if p.strip()] if path_str else []
                    file_name = input("Enter file name to register in folder: ")
                    local_path = input("Enter full local file path (e.g., /content/myfile.txt): ")
                    self.add_file_from_local_disk(folder_path, file_name, local_path)

                case '0':
                    print("Exiting...")
                    break

                case _:
                    print("Invalid choice. Please try again.")



tree = FolderTree()
tree.menu()



Menu:
1: Add Folder
2: Delete Folder
3: Search Folder
4: Add File
5: Open File
6: Show Last Opened File
7: Display Folder Tree
8: Add File from Local Disk
0: Exit
Enter your choice: 0
Exiting...
